# Produce `water_data.json` for the app's use

Authors: everett@bayes.org, mehdi@bayes.org

In [20]:
from collections import defaultdict
import datetime as datetime
import json
import random

import numpy as np
import pandas as pd

%matplotlib inline

DATA_DIR = "../data/"

### Load a reduced version of the water conservation dataset

#### This was manually edited by me (Everett) from the full dataset (also in the same folder), though in an ideal world I'd do it via a Makefile and code.

In [12]:
df = pd.read_csv(DATA_DIR + "uw_supplier_data110116_edited.csv", encoding="latin-1")
df = df.rename(columns=dict((c, c.strip()) for c in df.columns))
print("* " + "\n* ".join(df.columns))

* Supplier Name
* Mandatory Restrictions
* Reporting Month
* Total Population Served
* Conservation Standard (starting in June 2015) *Adjusted in March 2016 **Revised in June 2016
* REPORTED Total Monthly Potable Water Production Reporting Month
* REPORTED Monthly CII
* REPORTED Monthly Ag Use Reporting Month
* CALCULATED Total Monthly Potable Water Production Reporting Month Gallons
* CALCULATED Total Monthly Potable Water Production 2013 Gallons
* CALCULATED Monthly CII Reporting Month
* CALCULATED R-GPCD Reporting Month
* % Residential Use
* Hydrologic Region


In [13]:
def numberize(i):
    """Convert a string like '100,000' into a float, like 100000.0
    
    Since this data has a lot of variants for zero (missing data,
    null, strings like 'n/a'), we regard any failed float conversion
    as an indicator that the value is 0. This seems to be correct
    based on examining the data.
    """
    try:
        return float(str(i).replace(',', ''))
    except:
        return 0.0

In [14]:
# Rename wordy columns to more palatable strings
col_rename_dict = {
    'Supplier Name': 'supplier_name',
    'REPORTED Total Monthly Potable Water Production Reporting Month': 'reported_total_gal',
    'REPORTED Monthly CII': 'reported_cii_gal',
    'REPORTED Monthly Ag Use Reporting Month': 'reported_ag_gal',
    "CALCULATED Total Monthly Potable Water Production Reporting Month Gallons": "total_gal",
    "CALCULATED Monthly CII Reporting Month": "cii_gal",
    "CALCULATED R-GPCD Reporting Month": "r_gpcd",
    'Reporting Month': 'reporting_month',
    'Total Population Served': 'total_population_served',
    'REPORTED Residential Gallons-per-Capita-Day (R-GPCD) (starting in September 2014)': 'r_gpc',
    '% Residential Use': 'percent_residential_use',
    'Hydrologic Region': 'hydrologic_region',
    'Conservation Standard (starting in June 2015) *Adjusted in March 2016 **Revised in June 2016': 'conservation_standard'
}
cols = list(df.columns)
for c in cols:
    if c not in col_rename_dict:
        del df[c]
df = df.rename(columns=col_rename_dict)

In [15]:
# Compute a bunch of useful columns. Water usage breakdowns, etc.

df['reporting_month'] = pd.to_datetime(df['reporting_month'])
df['month'] = df['reporting_month'].apply(lambda x: x.month)
df['year'] = df['reporting_month'].apply(lambda x: x.year)

# Weirdly, the "total potable water reported" number INCLUDES agricultural
# water, whereas the "total potable gallons calculated" does NOT. The former
# is also reported in a range of units, while the latter converts to gallons.
# df['reported_ag_gal'] = df['reported_ag_gal'].apply(numberize)
# df['conversion_factor'] = df['total_gal'] / (df['reported_total_gal'] - df['reported_ag_gal'])
# df['ag_gal'] = df['reported_ag_gal'] * df['conversion_factor']
# df['person_days'] = df['r_gal'] / df['r_gpcd']
df['r_gal'] = df['total_gal'] * (df['percent_residential_use'] / 100.0)
df['cii_gal'] = df['cii_gal'].apply(numberize)
df['cii_gal_alternate'] = df['total_gal'] * (1 - df['percent_residential_use'] / 100.0)

df['conservation_standard'] = df['conservation_standard'].apply(
    lambda s: 0 if pd.isnull(s) else float(s.strip('%')) / 100.0)

df.head()

,supplier_name,reporting_month,total_population_served,conservation_standard,reported_total_gal,reported_cii_gal,reported_ag_gal,total_gal,cii_gal,r_gpcd,percent_residential_use,hydrologic_region,month,year,r_gal,cii_gal_alternate
0,East Bay Municipal Utilities District,2016-09-15,1400000,0.00,5636.2,912,NaN,5636200000,912000000,81.9,61,San Francisco Bay,9,2016,3438082000,2198118000
1,East Bay Municipal Utilities District,2016-08-15,1400000,0.00,6007.5,1141,NaN,6007500000,1141000000,84.4,61,San Francisco Bay,8,2016,3664575000,2342925000
2,East Bay Municipal Utilities District,2016-07-15,1400000,0.00,6056.6,994,NaN,6056600000,994000000,83.7,60,San Francisco Bay,7,2016,3633960000,2422640000
3,East Bay Municipal Utilities District,2016-06-15,1400000,0.00,5675.9,839,NaN,5675900000,839000000,82.4,61,San Francisco Bay,6,2016,3462299000,2213601000
4,East Bay Municipal Utilities District,2016-05-15,1400000,0.16,4959.3,955,NaN,4959300000,955000000,68.6,60,San Francisco Bay,5,2016,2975580000,1983720000


In [29]:
ebmud = df[df['supplier_name'] == 'East Bay Municipal Utilities District']
ebmud.shape

(28, 16)

In [38]:
computeUsage(ebmud)

0.8550202866446763


defaultdict(dict,
            {'commercialIndustrialPerCapita': {'2014-06': 0.0,
              '2014-07': 0.0,
              '2014-08': 0.0,
              '2014-09': 0.0,
              '2014-10': 0.0,
              '2014-11': 0.0,
              '2014-12': 0.0,
              '2015-01': 0.0,
              '2015-02': 0.0,
              '2015-03': 0.0,
              '2015-04': 0.0,
              '2015-05': 0.0,
              '2015-06': 653.8129496402878,
              '2015-07': 687.0503597122303,
              '2015-08': 717.410071942446,
              '2015-09': 720.5035971223022,
              '2015-10': 715.8273381294964,
              '2015-11': 609.3525179856115,
              '2015-12': 606.4748201438849,
              '2016-01': 857.8571428571429,
              '2016-02': 564.1379310714286,
              '2016-03': 578.5714285714286,
              '2016-04': 592.8571428571429,
              '2016-05': 682.1428571428571,
              '2016-06': 599.2857142857143,
              '201

In [56]:
def computeUsage(df):
    '''Given a dataframe of rows for a single provider, create usage dict.'''
    usage = defaultdict(dict)
    for i, row in df.iterrows():
        m = row['reporting_month'].strftime('%Y-%m')
        pop = float(row['total_population_served'])
        usage['totalPerCapita'][m] = numberize(row['total_gal']) / pop
        usage['residentialPerCapita'][m] = numberize(row['r_gal']) / pop
        #usage['agriculturalPerCapita'][m] = numberize(row['ag_gal']) / pop
        usage['commercialIndustrialPerCapita'][m] = numberize(row['cii_gal']) / pop
    
    # For demonstration purposes, add fake data for Oct 2016.
    # Will swap with real data once it's available.
    reference_month = '2015-10'
    if '2015-10' not in usage['totalPerCapita']:
        reference_month = random.choice(list(usage['totalPerCapita'].keys()))
    factor = 1 + random.uniform(0.1, 0.2) * random.choice([-1, 1])
    for k, v in usage.items():
        usage[k]['2016-10'] = usage[k][reference_month] * factor
    return dict(usage)  # Convert from defaultdict to regular dict

def computeMonthShare(usage, water_year):
    '''Compute the fraction of the year's usage that occurred each month.'''
    total = 0
    year = water_year
    month = 9
    share = {}
    for i in range(12):
        month_usage = usage['%d-%02d' % (year, month)]
        share[month] = month_usage
        total += month_usage
        month -= 1
        if month == 0:
            month = 12
            year -= 1
    for k in share.keys():
        share[k] /= total
    return share, total

def computePredictions(usage):
    '''Compute monthly total usage predictions for 2017.'''
    year = 2016
    month = 9
    predictions = {}
    for i in range(12):
        next_month = 12 if month == 1 else (month -1)
        next_year = year if next_month != 12 else (year - 1)
        
        current_key = '%d-%02d' % (year, month)
        last_year_key = '%d-%02d' % (year - 1, month)
        next_month_last_year_key = '%d-%02d' % (next_year - 1, next_month)
        if (current_key not in usage
            or last_year_key not in usage
            or next_month_last_year_key not in usage):
            return None
        
        growth = usage[current_key] / usage[last_year_key]
        predictions['%02d' %  next_month] = growth * usage[next_month_last_year_key]
        month = next_month
        year = next_year
    return predictions

In [57]:
# One dataset identifies water suppliers by ID, the other merely by name. We need to pair them.

providers = pd.read_csv(DATA_DIR + 'provider_ids.tsv', sep='\t')
provider_id_lookup = {}
for p, i in zip(providers['Provider'], providers['ID']):
    provider_id_lookup[p.lower()] = i

In [59]:
js = {}
for i, items in enumerate(df.groupby("supplier_name").groups.items()):
    name, indices = items
    print(i, name)
    name = name.lower()
    if name not in provider_id_lookup:
        print("Can't find supplier ID for '%s', skipping..." % name)
        continue

    supplier_df = df.loc[indices]
    usage = computeUsage(supplier_df)
    total = usage['totalPerCapita']
    del usage['totalPerCapita']
    preds = computePredictions(total)
    if not preds:
        print("Can't make predictions for '%s', skipping..." % name)
        continue

    month_share, year_total = computeMonthShare(total, 2016)
    supplier_df = supplier_df.set_index('reporting_month', drop=False)
    reduction = supplier_df.loc[datetime.datetime(2016, 9, 15),'conservation_standard']
    target = (1 - reduction) * year_total
    js[provider_id_lookup[name]] = {
        "agencyName": name,
        "totalUsage": total,
        "usageByCategory": usage,
        "monthlyPrediction": preds,
        "annualTarget": target,
    }

0 Golden State Water Company Cordova
1 Perris, City of
2 Castaic Lake Water Agency Santa Clarita Water Division
3 Humboldt Bay Municipal Water District
4 Covina  City of 
Can't find supplier ID for 'covina  city of ', skipping...
5 Palmdale Water District
6 Pleasanton  City of
7 Rio Vista, city of
8 East Palo Alto, City of
9 Nevada Irrigation District
10 California-American Water Company San Diego District
11 Placer County Water Agency
12 Santa Maria  City of
13 Inglewood  City of
14 Sacramento County Water Agency
15 Golden State Water Company Placentia
16 California Water Service Company Stockton
17 Rubidoux Community Service District
18 Golden State Water Company Bell-Bell Gardens
19 El Monte  City of
20 Burlingame  City of
21 Dublin San Ramon Services District
22 Helix Water District
23 California-American Water Ventura District
24 Manteca  City of
25 Menlo Park  City of
26 Golden State Water Company Claremont
27 Discovery Bay Community Services District
28 Alco Water Service
29 Wes

In [60]:
js[list(js.keys())[0]]

{'agencyName': 'turlock  city of',
 'annualTarget': 66274.898340529369,
 'monthlyPrediction': {'01': 4116.765314872664,
  '02': 2824.1374837745766,
  '03': 5551.819067297603,
  '04': 7786.256645374542,
  '05': 7107.631898992912,
  '06': 9725.70815050872,
  '07': 9574.388048978499,
  '08': 9047.919064820446,
  '09': 8317.500926713024,
  '10': 7255.460854239718,
  '11': 5963.979868223262,
  '12': 4092.710106948417},
 'totalUsage': {'2014-06': 11112.515980388025,
  '2014-07': 11182.759444233714,
  '2014-08': 9903.758601312147,
  '2014-09': 9952.09395765724,
  '2014-10': 8578.109849997149,
  '2014-11': 5802.110113653925,
  '2014-12': 4228.656523510487,
  '2015-01': 4190.725053033815,
  '2015-02': 4141.5546283418325,
  '2015-03': 6645.031679802194,
  '2015-04': 6993.017799693738,
  '2015-05': 6691.111392084967,
  '2015-06': 8489.307539933408,
  '2015-07': 8469.956870513199,
  '2015-08': 8375.971115887667,
  '2015-09': 7718.3518073643245,
  '2015-10': 7169.188407018727,
  '2015-11': 4907.489

In [61]:
with open(DATA_DIR + 'water_usage.json', 'w') as f:
    f.write(json.dumps(js, indent=4))